# Lista 1 - Spacy
**Nome:** Davy Alves de Souza

**Numero Usp:** 11561247


_________________________________________

O [spaCy]("https://spacy.io") é uma bilbioteca Python de código fonte [aberto]("https://github.com/explosion/spaCy") para Processamento de 
Linguagem Natural, constantemente a atualizada e mantida. Essa biblioteca é capaz de 
processar diversas línguas, inclusive o português brasileiro.

### Instalação



Após instalar o pacote spaCy (caso você tenha utilizado o arquivo `requirements.txt` ele já estará instalado) devemos baixar as ferramentas específicas para o português e para o inglês  com os seguintes comandos:

``` bash
$ python3 -m spacy download en
$ python3 -m spacy download pt
```

__Nota__:  omitir ‘\\$’. O símbolo ‘\\$’  é usado para indicar que se trata de um comando a ser digitado no terminal.

Uma vez que temos o pacote instalado e os módulos para português e inglês baixados,  podemos começar a utilizar  os spaCy, importando o pacote e carregando o módulo para português.

**Nota:** Caso seus resultados sejam diferentes dos descritos neste texto isso provavelmente significa que você está utilizando uma versão diferente do spaCy.

In [1]:
import spacy
# spacyPT = spacy.load('pt')
spacyPT = spacy.load('pt_core_news_lg')

In [2]:
spacy.__version__

'2.2.4'

É importante notar que o spaCy assume que os caracteres estão codificados no formato utf-8.  O primeiro passo portanto é gerar uma entrada nesse formato e e submetê-lo ao módulo carregado.

In [2]:
entrada = spacyPT("Mais vale um asno que me carregue que um cavalo que me derrube.")
entrada

Mais vale um asno que me carregue que um cavalo que me derrube.

### Tokenização (itemização)

A entrada que acabamos de gerar é uma sequência iterável de tokens (itens,  
ou instâncias de palavras). Se quisermos verificar qual o texto contido nessa 
sequência iterável,  usamos:

In [3]:
entrada.text

'Mais vale um asno que me carregue que um cavalo que me derrube.'

Se quisermos dividir a entrada em token,  podemos utilizar o método __split__: 

In [4]:
entrada.text.split()

['Mais',
 'vale',
 'um',
 'asno',
 'que',
 'me',
 'carregue',
 'que',
 'um',
 'cavalo',
 'que',
 'me',
 'derrube.']

Note que o ponto  final foi absorvido pela palavra;  o mesmo teria acontecido com 
outros sinais de pontuação a utilizar  o método __split__. Para separar a pontuação 
das palavras utilizamos a  tokenização implícita realizada pelo comando __in__:

In [5]:
[token for token in entrada]

[Mais, vale, um, asno, que, me, carregue, que, um, cavalo, que, me, derrube, .]

Note que os streams não estão entre aspas,  pois na realidade esta lista contém uma sequência de objetos da classe __Token__. 

Se o objetivo é obter uma lista de Strings,  podemos proceder da seguinte maneira.


In [6]:
[token.text for token in entrada]

['Mais',
 'vale',
 'um',
 'asno',
 'que',
 'me',
 'carregue',
 'que',
 'um',
 'cavalo',
 'que',
 'me',
 'derrube',
 '.']

E para eliminar totalmente a pontuação da lista,  é só restringirr  a sua criação usando __is_punct__.

In [7]:
[token.text for token in entrada if not token.is_punct]

['Mais',
 'vale',
 'um',
 'asno',
 'que',
 'me',
 'carregue',
 'que',
 'um',
 'cavalo',
 'que',
 'me',
 'derrube']

O spaCy já vem treinando para realizar etiquetagem morfossintática (PoS tagging),  o que pode ser mostrado da seguinte maneira.

In [8]:
[(token.text, token.pos_, spacy.explain(token.pos_)) for token in entrada]

[('Mais', 'ADV', 'adverb'),
 ('vale', 'VERB', 'verb'),
 ('um', 'DET', 'determiner'),
 ('asno', 'NOUN', 'noun'),
 ('que', 'PRON', 'pronoun'),
 ('me', 'PRON', 'pronoun'),
 ('carregue', 'VERB', 'verb'),
 ('que', 'SCONJ', 'subordinating conjunction'),
 ('um', 'DET', 'determiner'),
 ('cavalo', 'NOUN', 'noun'),
 ('que', 'PRON', 'pronoun'),
 ('me', 'PRON', 'pronoun'),
 ('derrube', 'VERB', 'verb'),
 ('.', 'PUNCT', 'punctuation')]

Note que ele foi capaz de identificar ao menos uma vez qual ocorrência da palavra `que`  é classificada como pronome relativo e qual ocorrência é uma conjunção complementizadora; mas há um erro também na classificação de uma das três tokens `que` (qual?).  Infelizmente não foi capaz de identificar que a palavra `asno` é um substantivo,  possivelmente porque essa palavra não pertence ao seu dicionário interno;  infelizmente também não temos como retreinar o etiquetador morfossintático do spaCy, em busca de maior precisão.  Neste casos só nos resta tentar implementar um outro etiquetador.

<font color="gray"><em>**Obs.:** A versão 2.3.2 do spaCy já está identificando a palavra **`asno`** como substantivo</em></font>

Mesmo assim,  a assistência do etiquetador nos permite fazer buscas bastante sofisticadas. Por exemplo podemos buscar os lemas de todos os verbos encontrados  na sentença.

In [9]:
[token.lemma_ for token in entrada if token.pos_ == 'VERB']

['valer', 'carregar', 'derrubar']

Os lemas de verbos conjugados nos fornecem  a sua forma infinitiva. Notem que o spacy tem dificuldade com o verbo mal-classificado `asno`.

### Reconhecimento de entidades nomeadas

A biblioteca já vem treinada com um  mecanismo que permite o reconhecimento de 
entidades mencionada (nomeadas). 

In [10]:
texto2 = spacyPT("O presidente Bolsonaro deu uma ordem ao Ministério do Meio Ambiente, que gerou calafrios no Congresso.")
print(texto2.ents)
[(entidade, entidade.label_, spacy.explain(entidade.label_)) for entidade in texto2.ents]

(Bolsonaro, Ministério do Meio Ambiente, Congresso)


[(Bolsonaro, 'PER', 'Named person or family.'),
 (Ministério do Meio Ambiente,
  'LOC',
  'Non-GPE locations, mountain ranges, bodies of water'),
 (Congresso, 'LOC', 'Non-GPE locations, mountain ranges, bodies of water')]

Note que ele acertou no reconhecimento das duas entidades nomeadas, deixando passar `Bolsonaro`, porém errou na 
classificação de todas;  as duas entidades reconhecidas são 
organizações.  Há bastante margem para melhorias de precisão com algoritmos mais 
robustos e treinados com muito mais dados.

___________________________
# <font color='blue'>  Questão 1 </font>

Utilizando o spacy, extraia o nome dos personagens presentes no terceiro capitulo da obra "Mémorias postumas de Brás Cubas" de Machado de Assis

In [22]:
cap_3_bras_cubas = "Mas, já que falei nos meus dois tios, deixem-me fazer aqui um curto esboço genealógico.        O fundador de minha família foi um certo Damião Cubas, que floresceu na primeira metade do século XVIII. Era tanoeiro de ofício, natural do Rio de Janeiro, onde teria morrido na penúria e na obscuridade, se somente exercesse a tanoaria. Mas não; fez-se lavrador, plantou, colheu, permutou o seu produto por boas e honradas patacas, até que morreu, deixando grosso cabedal a um filho, o licenciado Luís Cubas. Neste rapaz é que verdadeiramente começa a série de meus avós -- dos avós que a minha família sempre confessou -  porque o Damião Cubas era afinal de contas um tanoeiro, e talvez mau tanoeiro, ao passo que o Luís Cubas estudou em Coimbra, primou no Estado, e foi um dos amigos particulares do vice-rei conde da Cunha.        Como este apelido de Cubas lhe cheirasse excessivamente a tanoaria, alegava meu pai, bisneto do Damião, que o dito apelido fora dado a um cavaleiro, herói nas jornadas da Africa, em prêmio da façanha que praticou arrebatando trezentas cubas ao mouros. Meu pai era homem de imaginação; escapou à tanoaria nas asas de um calembour. Era um bom caráter, meu pai, varão digno e leal como poucos. Tinha, é verdade, uns fumos de pacholice; mas quem não é um pouco pachola nesse mundo? Releva notar que ele não recorreu à inventiva senão depois de experimentar a falsificação; primeiramente, entroncou-se na família daquele meu famoso homônimo, o capitão-mor Brás Cubas, que fundou a vila de São Vicente, onde morreu em 1592, e por esse motivo é que me deu o nome de Brás. Opôs-se-lhe, porém, a família do capitão-mor, e foi então que ele imaginou as trezentas cubas mouriscas.        Vivem ainda alguns membros de minha família, minha sobrinha Venância, por exemplo, o lírio-do-vale, que é a flor das damas do seu tempo; vive o pai, o Cotrim, um sujeito que... Mas não antecipemos os sucessos; acabemos de uma vez com o nosso emplasto. "
cap_3_bras_cubas

'Mas, já que falei nos meus dois tios, deixem-me fazer aqui um curto esboço genealógico.        O fundador de minha família foi um certo Damião Cubas, que floresceu na primeira metade do século XVIII. Era tanoeiro de ofício, natural do Rio de Janeiro, onde teria morrido na penúria e na obscuridade, se somente exercesse a tanoaria. Mas não; fez-se lavrador, plantou, colheu, permutou o seu produto por boas e honradas patacas, até que morreu, deixando grosso cabedal a um filho, o licenciado Luís Cubas. Neste rapaz é que verdadeiramente começa a série de meus avós -- dos avós que a minha família sempre confessou -  porque o Damião Cubas era afinal de contas um tanoeiro, e talvez mau tanoeiro, ao passo que o Luís Cubas estudou em Coimbra, primou no Estado, e foi um dos amigos particulares do vice-rei conde da Cunha.        Como este apelido de Cubas lhe cheirasse excessivamente a tanoaria, alegava meu pai, bisneto do Damião, que o dito apelido fora dado a um cavaleiro, herói nas jornadas da

In [ ]:
#Seu código aqui

In [23]:
# Seu código aqui

bras_cubas_txt = spacyPT(cap_3_bras_cubas)
# print(bras_cubas_txt.ents)
[entidade for entidade in bras_cubas_txt.ents if entidade.label_ == 'PER']

[Damião Cubas,
 Luís Cubas,
 Damião Cubas,
 Luís Cubas,
 conde da Cunha,
 Cubas,
 Damião,
 Brás Cubas,
 Venância,
 Cotrim]

Quais destas repostas estão corretas?  Quais personagens estão faltando?

**RESP.:** <em>Podemos notar que o personagem Damião apareceu três vezes na lista (duas como Damião Cubas e mais uma como Damião). Luís Cubas também foi duplicado. Outro erro que podemos notar é que 'Cunha' foi identificado como 'conde da Cunha'. Por fim, ele não conseguiu identificar 'Brás' (que não é a mesma pessoa que 'Brás Cubas').</em>

# <font color='blue'>  Questão 2 </font>

Extraia todos os pronomes deste capitulo.

_____________
**<font color='red'> Sua resposta aqui </font>**


In [24]:
#Seu código aqui

[(token.text, token.pos_, spacy.explain(token.pos_)) for token in bras_cubas_txt if token.pos_ == 'PRON']

[('que', 'PRON', 'pronoun'),
 ('onde', 'PRON', 'pronoun'),
 ('que', 'PRON', 'pronoun'),
 ('lhe', 'PRON', 'pronoun'),
 ('que', 'PRON', 'pronoun'),
 ('que', 'PRON', 'pronoun'),
 ('poucos', 'PRON', 'pronoun'),
 ('quem', 'PRON', 'pronoun'),
 ('ele', 'PRON', 'pronoun'),
 ('que', 'PRON', 'pronoun'),
 ('onde', 'PRON', 'pronoun'),
 ('me', 'PRON', 'pronoun'),
 ('Opôs-se-lhe', 'PRON', 'pronoun'),
 ('ele', 'PRON', 'pronoun'),
 ('que', 'PRON', 'pronoun'),
 ('que', 'PRON', 'pronoun')]

# <font color='blue'>  Questão 2 </font>

Extraia todos os pronomes deste capitulo.

_____________
**<font color='red'> Sua resposta aqui </font>**


# <font color='blue'>  Questão 3 </font>
Utilize os visualizadores para explorar o mapa de dependencias de uma frase a sua escolha deste capitulo.
https://spacy.io/usage/visualizers

Você pode acessar diretamente uma frase especifica ao utilizar o gerador "sents", por exemplo:

```python
frases = [frase for frase in texto.sents]
frases[2]


Era tanoeiro de ofício, natural do Rio de Janeiro LOC , onde teria morrido na penúria e na obscuridade, se somente exercesse a tanoaria. 
```


_______________


**<font color='red'> Sua resposta aqui </font>**

In [37]:
from spacy import displacy

# Seu código aqui
frases = [frase for frase in bras_cubas_txt.sents]

# Primeira frase
colors = {"LOC": "linear-gradient(90deg, #aa9cfc, #fc9ce7)"}
options = {"ents": ["LOC"], "colors": colors}
displacy.render(frases[2], style="ent", jupyter=True, options=options)

In [58]:
colors = {"LOC": "linear-gradient(90deg, #aa9cfc, #fc9ce7)", "PER": "linear-gradient(90deg, #ccffaa, #449cff)"}
options = {"ents": ["PER", "LOC"], "colors": colors}
displacy.render(frases[12], style="ent", jupyter=True, options=options)

In [52]:
options = {"compact": True, "bg": "#09a3d5", "color": "white"}
displacy.render(frases[2], style="dep", options=options)

# Fontes 
Tanto o capitulo utilizado nesta aula quanto a obra completa fazem parte do dominio publico e podem ser encontrados em http://www.dominiopublico.gov.br/download/texto/bv000215.pdf